In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext wurlitzer

In [3]:
import pandas as pd
import numpy as np
import logging
logging.basicConfig(level = logging.INFO)

In [4]:
import pyfdb
import findlibs
import yaml
from pathlib import Path
import os
import pandas as pd
import pyodc
import shutil

In [5]:
from pathlib import Path
from ionbeam.core.config_parser import parse_config
from ionbeam.core.bases import *
from IPython.display import display

config_file = Path("~/git/IonBeam_bundle/IonBeam/config/").expanduser()
config, actions = parse_config(config_file,
                    environment  = "local",
                    sources = ["acronet"]
                    )

INFO:ionbeam.sources.cima.cima:Looking for cache file at /Users/math/git/IonBeam_bundle/data/cache/cima_cache.pickle
INFO:ionbeam.sources.cima.cima:Loaded cache file /Users/math/git/IonBeam_bundle/data/cache/cima_cache.pickle


In [6]:
from ionbeam.metadata.db import init_db
import sqlalchemy
Base = sqlalchemy.orm.declarative_base()
force_init = False

# if force_init or Base.metadata.reflect(config.globals.sql_engine) is None:
#     init_db(config.globals)

In [7]:
from uuid import UUID

id2action = {a.id : a for a in actions}
sources = [a for a in actions if isinstance(a, Source)]
processors = [a for a in actions if not isinstance(a, Source)]

chains = [[s,] for s in sources]
for a in processors:
    if not isinstance(a.match, UUID): 
        chains.append([str(a.match), a,])
        continue
    for c in chains:
        matches = a.match.int == c[-1].id.int
        if matches: c.append(a)
            

for c in chains:
    print(" --> ".join(str(a) for a in c))

AcronetSource --> DropEmpty --> ExtractMetaData --> CanonicaliseColumns --> AddAcronetMetadata --> ComputeDateTime --> ComputeMARSIdentifier --> SimpleODCEncoder
[Match(state = 'odc_encoded')] --> FDBWriter()


In [8]:
sources, stateless_actions, aggregators = [], [], []
for action in actions:
    if isinstance(action, Source):
        sources.append(action)
    elif isinstance(action, Aggregator):
        aggregators.append(action)
    else:
        stateless_actions.append(action)

print("Sources")
for i, a in enumerate(sources):
    display(a)

print("Aggregators")
for i, a in enumerate(aggregators):
    display(a)

print("Actions")
for i, a in enumerate(stateless_actions):
    display(a)

Sources


id,85f5556f-33a9-4760-805f-fed3c128f392
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/acronet/actions.yaml
mappings,"[RawVariable(name='time', key='time', type='datetime', unit=None, discard=False, canonical_variab..."
finish_after,None
cache_version,3
use_cache,True
source,acronet
maximum_request_size,"10 days, 0:00:00"
minimum_request_size,0:05:00
name,value
source_action_id,85f5556f-33a9-4760-805f-fed3c128f392


Aggregators
Actions


id,aa060fc5-4b2e-4b8f-acac-5306ffaf8a59
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/acronet/actions.yaml
match,85f5556f-33a9-4760-805f-fed3c128f392
name,value
source_action_id,aa060fc5-4b2e-4b8f-acac-5306ffaf8a59
state,parsed
mars_id,{}
name,value
canonical_variables,"[CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the obse..."
data_path,/Users/math/git/IonBeam_bundle/data
cache_path,/Users/math/git/IonBeam_bundle/data/cache


id,d9ab7ff6-59dc-463a-8a80-b59e54e4704c
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/acronet/actions.yaml
match,aa060fc5-4b2e-4b8f-acac-5306ffaf8a59
copy_metadata_to_columns,"{'station_id': 'station.id', 'station_name': 'station.name', 'lat': 'station.lat', 'lon': 'statio..."
name,value
source_action_id,d9ab7ff6-59dc-463a-8a80-b59e54e4704c
state,parsed
mars_id,{}
name,value
canonical_variables,"[CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the obse..."
data_path,/Users/math/git/IonBeam_bundle/data


...

id,d0348bce-49a5-4b62-8fae-9239d798a277
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/acronet/actions.yaml
match,b541f124-8379-4163-bdc9-39a15e90ca8c
name,value
source_action_id,d0348bce-49a5-4b62-8fae-9239d798a277
state,parsed
mars_id,{}
name,value
canonical_variables,"[CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the obse..."
data_path,/Users/math/git/IonBeam_bundle/data
cache_path,/Users/math/git/IonBeam_bundle/data/cache


id,aaad17e9-a6c1-4e3f-b507-1091ad05a33d
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/acronet/actions.yaml
match,d0348bce-49a5-4b62-8fae-9239d798a277
name,value
source_action_id,aaad17e9-a6c1-4e3f-b507-1091ad05a33d
state,parsed
mars_id,{}
name,value
canonical_variables,"[CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the obse..."
data_path,/Users/math/git/IonBeam_bundle/data
cache_path,/Users/math/git/IonBeam_bundle/data/cache


id,0041bae3-c108-4686-9105-93c82d41b1b9
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/acronet/actions.yaml
match,aaad17e9-a6c1-4e3f-b507-1091ad05a33d
lookup,"{'class': 'const.rd', 'expver': 'const.xxxx', 'stream': 'const.lwda', 'aggregation_type': 'const...."
name,value
source_action_id,0041bae3-c108-4686-9105-93c82d41b1b9
state,parsed
mars_id,{}
name,value
canonical_variables,"[CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the obse..."
data_path,/Users/math/git/IonBeam_bundle/data


id,17b97404-a3bc-4720-9ac4-04da8a4147ed
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/acronet/actions.yaml
match,0041bae3-c108-4686-9105-93c82d41b1b9
output,outputs/{source}/odb/{observation_variable}/{observation_variable}_{time_span.start}.odb
name,value
source_action_id,17b97404-a3bc-4720-9ac4-04da8a4147ed
state,odc_encoded
mars_id,{}
name,value
canonical_variables,"[CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the obse..."
data_path,/Users/math/git/IonBeam_bundle/data


id,439ff392-fb7b-4776-9fe2-1d3dfca3b2b7
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/actions.yaml
match,[Match(state = 'odc_encoded')]
config,"{'engine': 'toc', 'spaces': [{'handler': 'Default', 'roots': [{'path': '/Users/math/git/IonBeam_b..."
debug,[]
name,value
source_action_id,439ff392-fb7b-4776-9fe2-1d3dfca3b2b7
state,written
mars_id,{}
name,value
canonical_variables,"[CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the obse..."


In [9]:
online = True
if online:
    source = sources[0]
    source.globals.offline = !online
    source.use_cache = True
    incoming_message_stream = source.generate()
    message = next(incoming_message_stream)
    message
else:
    p = Path('/Users/math/git/IonBeam-Deployment/data/inputs/meteotracker/MeteoTracker_62ae154f1d8e11061d4474b2.csv')
    message = FileMessage(
        metadata=MetaData(state='raw',
            source='meteotracker',
            filepath= p,
            variables = list(pd.read_csv(p).columns),             
            ))


INFO:ionbeam.sources.cima.cima:Fetching the token...


In [10]:
from ionbeam.core.bases import FinishMessage, FileMessage, MetaData
from ionbeam.aggregators import TimeAggregator

possible_actions = [a for a in actions if not isinstance(a, Source)]

message_history = []
while True:
    display(message)
    message_history.append(message)
    matching = [action for action in possible_actions if action.matches(message)]
    
    if not matching: 
        print("No more matches")
        break

    print("That matched with: ", [str(a) for a in matching])
    action = matching[0]

    # Special case for the TimeAggregator
    if action.__class__.__name__ == "TimeAggregator":
        list(action.process(message)) #need the list call here to pump the iterator to completion even if it doesn't return anything
        message = next(action.process(FinishMessage("We're done!")))
    else:
        message = next(action.process(message))
        

TabularMessage(metadata=MetaData(source_action_id=UUID('85f5556f-33a9-4760-805f-fed3c128f392'), state='raw', source=None, observation_variable=None, time_span=TimeSpan(start=datetime.datetime(2025, 1, 2, 17, 0, tzinfo=datetime.timezone.utc), end=datetime.datetime(2025, 1, 2, 17, 5, tzinfo=datetime.timezone.utc)), encoded_format=None, filepath=None, mars_id={}, unstructured={'station': {'name': 'Casa della Miniera', 'id': 'casa_della_miniera', 'lat': 44.427803, 'lon': 8.596518, 'sensors': {'PLUVIOMETRO': {'unit': 'mm', 'id': '-1937152652_2'}, 'TERMOMETRO': {'unit': '°C', 'id': '210331000_2'}, 'IGROMETRO': {'unit': '%', 'id': '210331001_2'}, 'DIREZIONEVENTO': {'unit': 'Degrees', 'id': '210824571_2'}, 'ANEMOMETRO': {'unit': 'KTS', 'id': '210331003_2'}, 'BAROMETRO': {'unit': 'hPa', 'id': '210331007_2'}, 'BATTERIA': {'unit': 'V', 'id': '210330997_2'}, 'TERMOMETRO_INTERNA': {'unit': '°C', 'id': '210331006_2'}, 'DIREZIONEVENTO_RAFFICA': {'unit': 'Degrees', 'id': '210824573_2'}, 'ANEMOMETRO_RAFFICA': {'unit': 'KTS', 'id': '210331005_2'}, 'SIGNAL_STRENGTH': {'unit': 'CSQ', 'id': '210330999_2'}}}, 'author': 'acronet'}, internal_id=None, external_id=None, date=None, time=None), history=[], data=   PLUVIOMETRO [mm]                      time  TERMOMETRO [°C]  IGROMETRO [%]  \
0               0.0 2025-01-02 17:03:00+00:00              5.0           86.5   
1               0.0 2025-01-02 17:04:00+00:00              5.0           86.4   

   DIREZIONEVENTO [Degrees]  ANEMOMETRO [m/s]  BAROMETRO [hPa]  BATTERIA [V]  \
0                 223.40001              11.3        895.70996         12.59   
1                 211.80000              12.0        895.68000         12.62   

   TERMOMETRO_INTERNA [°C]  ANEMOMETRO_RAFFICA [m/s]  SIGNAL_STRENGTH [CSQ]  
0                      5.9                 13.100000                    NaN  
1                      5.9                 13.400001                    NaN  , columns={})

That matched with:  ['DropEmpty']


name,value
source_action_id,aa060fc5-4b2e-4b8f-acac-5306ffaf8a59
state,parsed
time_span,2025-01-02T17:00:00+00:00 - 2025-01-02T17:05:00+00:00
mars_id,{}
unstructured,"{'station': {'name': 'Casa della Miniera', 'id': 'casa_della_miniera', 'lat': 44.427803, 'lon': 8..."
name,value
name,TabularMessage
metadata,"MetaData(source = None, variable = None)"
name,value
name,DropEmpty


That matched with:  ['ExtractMetaData']


name,value
source_action_id,d9ab7ff6-59dc-463a-8a80-b59e54e4704c
state,parsed
time_span,2025-01-02T17:00:00+00:00 - 2025-01-02T17:05:00+00:00
mars_id,{}
unstructured,"{'station': {'name': 'Casa della Miniera', 'id': 'casa_della_miniera', 'lat': 44.427803, 'lon': 8..."
name,value
name,TabularMessage
metadata,"MetaData(source = None, variable = None)"
name,value
name,DropEmpty


...

That matched with:  ['SimpleODCEncoder']


name,value
source_action_id,17b97404-a3bc-4720-9ac4-04da8a4147ed
state,odc_encoded
time_span,2025-01-02T17:00:00+00:00 - 2025-01-02T17:05:00+00:00
encoded_format,odb
mars_id,"{'class': Key(key='class', value='rd', key_spec=class, reason='Matches', odb_table=None), 'expver..."
unstructured,"{'station': {'name': 'Casa della Miniera', 'id': 'casa_della_miniera', 'lat': 44.427803, 'lon': 8..."
internal_id,00d541a3ff24897b
external_id,casa_della_miniera
date,20250102
time,1700


That matched with:  ['FDBWriter()']


name,value
source_action_id,439ff392-fb7b-4776-9fe2-1d3dfca3b2b7
state,written
time_span,2025-01-02T17:00:00+00:00 - 2025-01-02T17:05:00+00:00
encoded_format,odb
mars_id,"{'class': Key(key='class', value='rd', key_spec=class, reason='Matches', odb_table=None), 'expver..."
unstructured,"{'station': {'name': 'Casa della Miniera', 'id': 'casa_della_miniera', 'lat': 44.427803, 'lon': 8..."
internal_id,00d541a3ff24897b
external_id,casa_della_miniera
date,20250102
time,1700


No more matches


## Reload and retry failed action

In [11]:
from ionbeam.core.config_parser import reload_action

print(f"Failed action: {action} Definition: {action.definition_path}")

config, action = reload_action(config, action)

message = next(action.process(message_history[-1]))

message

Failed action: FDBWriter() Definition: /Users/math/git/IonBeam_bundle/IonBeam/config/actions.yaml


ValueError: Unexpected message type <class 'ionbeam.core.bases.DataMessage'>

In [ ]:
m = message_history[-1]
m.metadata.unstructured

In [ ]:
import pyodc as pyodc
from io import BytesIO

f = BytesIO(message.bytes)
df = pyodc.read_odb(f, single = True)
df

In [ ]:
from ionbeam.core.converters import unit_conversions

df = pd.DataFrame({"A" : np.arange(100), "B": np.linspace(0,1,100)})

In [ ]:
converter = unit_conversions["°C -> K"]


%timeit df.apply({"A" : converter}, raw = True, by_row = False)

In [ ]:
converter = lambda t: t + 273.15
converter2 = lambda t: t * 1000

df = pd.DataFrame({"A" : np.arange(100), "B": np.linspace(0,1,100), "C" : np.arange(100)})

df = df.apply({"A" : converter, "B" : converter2}, raw = True, by_row = False)

df

In [ ]:
converter = lambda 